In [4]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
pyspark.__version__

'3.3.2'

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-06 12:45:48--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.130, 13.226.36.73, 13.226.36.218, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  59.5MB/s    in 1.0s    

2025-03-06 12:45:49 (59.5 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("homework-5") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/06 17:24:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
!ls

data_ingestion.ipynb  yellow_tripdata_2024-10.parquet


In [6]:
file_path = "yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(file_path).repartition(4)

In [4]:
rep_output = df.write.parquet(f"rep/rep-{file_path}")

AnalysisException: path file:/home/nicolas/notebooks/week_5/homework/rep/rep-yellow_tripdata_2024-10.parquet already exists.

In [14]:
!du -h yellow_tripdata_2024-10.parquet

62M	yellow_tripdata_2024-10.parquet


In [16]:
!du -h rep/rep-yellow_tripdata_2024-10.parquet

98M	rep/rep-yellow_tripdata_2024-10.parquet


In [17]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [18]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [20]:
df.select('tpep_pickup_datetime') \
  .show(5)

+--------------------+
|tpep_pickup_datetime|
+--------------------+
| 2024-10-01 22:46:41|
| 2024-10-09 22:11:58|
| 2024-10-03 08:41:40|
| 2024-10-09 13:56:51|
| 2024-10-09 10:36:09|
+--------------------+
only showing top 5 rows



In [7]:
df.createOrReplaceTempView("yellow_trips")

In [31]:
spark.sql('''select *
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and payment_type <= 0
             ''')\
      .show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-15 20:03:15|  2024-10-15 20:21:05|           null|         3.49|      null|              null|         162|         263|           0|      18.66|  0.0|    0.5|       0.

In [30]:
spark.sql('''select *
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and RatecodeID <= 0
             ''')\
      .show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---------

In [32]:
spark.sql('''select *
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and store_and_fwd_flag not in ('Y', 'N')
             ''')\
      .show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---------

In [44]:
spark.sql('''select total_amount, passenger_count
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and fare_amount < 0
             and payment_type in (1, 2, 3, 4)
             and passenger_count = 0
             ''')\
      .show()

+------------+---------------+
|total_amount|passenger_count|
+------------+---------------+
+------------+---------------+



In [8]:
spark.sql('''select distinct trip_distance
             from yellow_trips
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and trip_distance <=0
             ''')\
      .show()

+-------------+
|trip_distance|
+-------------+
|          0.0|
+-------------+



In [26]:
spark.sql('''select COUNT(1)
             from yellow_trips
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and tpep_pickup_datetime <= tpep_dropoff_datetime
             ''')\
      .show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [28]:
spark.sql('''select COUNT(1)
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and payment_type != 0
             ''')\
      .show()

+--------+
|count(1)|
+--------+
|  115133|
+--------+



In [10]:
spark.sql('''select COUNT(1)
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             ''')\
      .show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [21]:
spark.sql('''select COUNT(PULocationID)
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and PULocationID is NULL
             ''')\
      .show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---------

In [43]:
spark.sql('''select COUNT(1)
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and CAST(tpep_dropoff_datetime AS DATE) >= '2024-10-15'
             and fare_amount > 0
             ''') \
      .show()

+--------+
|count(1)|
+--------+
|  126164|
+--------+



In [14]:
import pandas as pd
import pyarrow.parquet as pa

In [15]:
pandas_df = pd.read_parquet(file_path)

In [16]:
pandas_df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
dtype: object

In [23]:
pandas_df_oct_15 = pandas_df[(pandas_df["tpep_pickup_datetime"] >= "2024-10-15 00:00:00") & (pandas_df["tpep_pickup_datetime"] <= "2024-10-15 23:59:59")]

In [19]:
len(pandas_df_oct_15.index)

128893

In [61]:
spark.sql('''select tpep_pickup_datetime, tpep_dropoff_datetime, TIMESTAMPDIFF(MINUTE,tpep_pickup_datetime, tpep_dropoff_datetime) as duration
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             and CAST(tpep_dropoff_datetime AS DATE) >= '2024-10-15'
             and fare_amount > 0
             and trip_distance > 0
             and TIMESTAMPDIFF(MINUTE,tpep_pickup_datetime, tpep_dropoff_datetime) < 0
             ''') \
      .show()

+--------------------+---------------------+--------+
|tpep_pickup_datetime|tpep_dropoff_datetime|duration|
+--------------------+---------------------+--------+
+--------------------+---------------------+--------+



In [14]:
spark.sql('''select distinct payment_type
             from yellow_trips 
             where CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
             ''')\
      .show()

+------------+
|payment_type|
+------------+
|           0|
|           1|
|           3|
|           2|
|           4|
+------------+



In [35]:
spark.sql('''select MAX(TIMESTAMPDIFF(HOUR,tpep_pickup_datetime, tpep_dropoff_datetime))
             from yellow_trips
             where CAST(tpep_pickup_datetime AS DATE) >= '2024-10-01' and
             CAST(tpep_pickup_datetime AS DATE) <= '2024-10-31'
             ''')\
      .show()

+---------------------------------------------------------------------+
|max(timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime))|
+---------------------------------------------------------------------+
|                                                                  162|
+---------------------------------------------------------------------+



In [36]:
#Question 5
#Since i'm not running pyspark on standalone mode, the jobs UI runs on port 4040 by default

In [37]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-06 13:35:47--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.36.130, 13.226.36.196, 13.226.36.73, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.36.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-06 13:35:47 (126 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [8]:
lookup_file_path = "taxi_zone_lookup.csv"
df_lookup = spark.read.option("header",True) \
            .csv(lookup_file_path)

In [47]:
df_lookup.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [9]:
df_lookup.createOrReplaceTempView("zones")

In [51]:
spark.sql('''select zones.Zone,
                    yellow_trips.PULocationID,
                    COUNT(yellow_trips.PULocationID) as trips
             from yellow_trips, zones
             where yellow_trips.PULocationID = zones.LocationID
             group by zones.Zone, yellow_trips.PULocationID
             order by trips ASC
          ''') \
          .show()

+--------------------+------------+-----+
|                Zone|PULocationID|trips|
+--------------------+------------+-----+
|Governor's Island...|         105|    1|
|       Rikers Island|         199|    2|
|       Arden Heights|           5|    2|
|         Jamaica Bay|           2|    3|
| Green-Wood Cemetery|         111|    3|
|Charleston/Totten...|          44|    4|
|       West Brighton|         245|    4|
|       Port Richmond|         187|    4|
|Eltingville/Annad...|          84|    4|
|   Rossville/Woodrow|         204|    4|
|        Crotona Park|          59|    6|
|         Great Kills|         109|    6|
|Heartland Village...|         118|    7|
|     Mariners Harbor|         156|    7|
|Saint George/New ...|         206|    9|
|             Oakwood|         176|    9|
|       Broad Channel|          30|   10|
|New Dorp/Midland ...|         172|   10|
|     Pelham Bay Park|         184|   12|
|         Westerleigh|         251|   12|
+--------------------+------------

In [68]:
spark.sql('''select count(1)
             from yellow_trips
             where PULocationID = 105
          ''') \
          .show()

+--------+
|count(1)|
+--------+
|       1|
+--------+

